# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [3]:
import PyPDF2 as pyf
from pathlib import Path

nome = 'MGLU_ER_3T20_POR.pdf'
arquivo_pdf = pyf.PdfReader(nome)

for i, pagina in enumerate(arquivo_pdf.pages):
    num_pagina = i + 1
    novo_pdf = pyf.PdfWriter()
    novo_pdf.add_page(pagina)
    with Path(f'paginas/Arquivo Pagina {num_pagina}.pdf').open(mode='wb') as arquivo:
        novo_pdf.write(arquivo)



#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

### Extra: Para adicionar todas as páginas de 2 pdfs

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

- Rodar Página

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [21]:
arquivo = pyf.PdfReader(r"C:\Users\anton\Downloads\zoneamento\CAM2024084848-240306165204.PDF")

qtd_pag = len(arquivo.pages)
print(qtd_pag)

metadados_arquivo = arquivo.metadata
print(metadados_arquivo)

5
{'/Creator': '', '/CreationDate': 'D:20240306', '/Title': '', '/Author': '', '/Producer': 'TExtraDevices', '/Keywords': '', '/Subject': ''}


In [29]:
texto_ref = 'Área do  Terreno:'

for i, pagina in enumerate(arquivo.pages):
    texto_pag = pagina.extract_text()
    if texto_ref in texto_pag:
        print('Numero Pagina: ', i + 1)
        texto_analisar = texto_pag

Numero Pagina:  5


In [30]:
print(texto_analisar)

CONSULTA INFORMATIVA DO LOTE
 INDICAÇÃO FISCAL BLOQUEADA
 Inscrição Imobiliária
 Sublote
 Nº da Consulta / Ano
 Indicação Fiscal
 11.015.018
 02.0.0054.0258.00-3
 -
 84848/2024
 PREFEITURA MUNICIPAL DE CURITIBA
 Secretaria Municipal do Urbanismo
Área Total Construída: Área do  Terreno: 250,00 m²
Dados dos Sublotes
 Sublote
Utilização Ano Construção
 Área Construída
 Comercial
 0000
 1913
 250,00 m²
Qtde. de Sublotes: 1 413,00 m²
Cód. Logradouro Planta
Pavimentação Esgoto Iluminação Pública Coleta de LixoInfraestrutura Básica
D
ASFALTO
EXISTE
N012 Sim
 Sim
Bacia(s) Hidrográfica(s)
BACIA BELEM Principal
Observações Gerais
1 - Considerando a necessidade de adequar e organizar os espaços destinados a circulação de pedestres, a construção ou 
reconstrução de passeios deverá obedecer os padrões definidos pelo Decreto 1.066/2006
2 - A altura da edificação deverá obedecer as restrições do Ministério da Aeronaútica, referentes ao plano da zona de 
proteção dos aeródromos e as restrições da Agên

#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)
    
    - Cuidado 1: Instale o tabula-py (não instale o tabula). Se instalar o tabula errado, desinstale ele, instale o tabula-py, desinstale o tabula-py e instale novamente o tabula-py. Reinicie o kernel do Jupyter após isso
    
    - Cuidado 2: Tem que ter o java instalado no seu computador (depois de instalar, reinicie o computador)

In [17]:
import tabula

tabelas = tabula.read_pdf('MGLU_ER_3T20_POR.pdf', pages=5)
df_resultado = tabelas[0]
# excluir linhas totalmente vazias
df_resultado = df_resultado.dropna(how='all', axis=0)
# excluir colunas totalmente vazias
df_resultado = df_resultado.dropna(how='all', axis=1)
df_resultado.columns = df_resultado.iloc[0]
df_resultado = df_resultado.iloc[1:]
df_resultado = df_resultado.reset_index(drop=True)

display(df_resultado)

c:\Users\anton\AppData\Local\Programs\Python\Python312\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


,R$ milhões (exceto quando indicado),3T2,3T1,Var(%,9M2,9M1,Var(%
0,Vendas Totais1 (incluindo marketplace),"12.355,5","6.817,6","81,2","28.584,","18.282,6","56,3"
1,Receita Bruta,"10.349,5","5.999,4","72,5","23.652,","16.508,8","43,3"
2,Receita Líquida,"8.308,3","4.864,2","70,8","19.111,","13.501,3","41,6"
3,Lucro Bruto,"2.178,7","1.424,9","52,9","5.034,","3.728,6","35,0"
4,Margem Bruta,"26,2","29,3","-3,1 p","26,3","27,6","-1,3 p"
5,EBITDA,"546,1","501,2","9,0","1.022,","1.276,5","-19,9"
6,Margem EBITDA,"6,6","10,3","-3,7 p","5,3","9,5","-4,2 p"
7,Lucro Líquido,"206,0","235,1","-12,4","172,","753,8","-77,2"
8,Margem Líquida,"2,5","4,8","-2,3 p","0,9","5,6","-4,7 p"
9,Lucro Bruto - Ajustado,"2.178,7","1.488,9","46,3","5.034,","3.964,6","27,0"


#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url